# Damage function fit

In this notebook we fit the damage function for Burke *et al.*, 2018, Rose *et al.*, 2017, and Howard & Sterner, 2017. I have the data from Burke _et al._, 2018 using their reproduction code (thanks David!), I graph grabber'd the data for the second damage function from the IPCC WGII plot on p. 2880, and the Howard & Sterner data was taken from a figure in their appendix. I then fit the data to a quadratic function.

## Setup

In [ ]:
import sys
import datetime
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from scipy.optimize import curve_fit
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator

mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'

# general plot parameters
color_list = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7']
marker_list = ['o', 's', 'P', '+', 'D', 'v', '3']
fontsize = 22
labelsize = 20
markersize = 10
figsize = (16, 10)

# plotting parameters
color_list = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7'] * 2
marker_list = ['o', 's', 'P', '*', '<', 'v', '>', '^', 'p']
linestyle_list = ['solid', 'dashed', 'dashdot', 'dotted'] * 4
linewidth = 2
node_times = [2020, 2030, 2060, 2100, 2150, 2200]
x_label = 'Year'

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
plt.rcParams['figure.figsize'] = 16, 10
params = {'legend.fontsize': 18,
          'legend.frameon': True,
          'figure.figsize': (16, 10),
         'axes.labelsize': 20,
         'axes.titlesize': 20,
         'xtick.labelsize': 18,
         'ytick.labelsize': 18,
         'font.family': 'serif'}
plt.rcParams.update(params)

# make base filename
today = datetime.datetime.now()
year = str(today.year)
day = str(today.day)
month = str(today.month)

basefile = ''.join([month, '-', day, '-', year, '-'])
path = '../data/' # might need to be edited for a given user's needs/directory structure. just point the notebook towards the 'data' directory.

## General fitting function

Each one of our damage functions follow the same damage function fitting and uncertainty estimation.

In [ ]:
def fitted_curve(loc, scale, conc_mult):
    """Get fitted polynomial coefficients.
    
    Parameters
    ----------
    loc: float
        guess for mean curve fit parameter D_1d
    
    scale: float
        guess for std deviaion of curve fit parameter D_1d
    
    conc_mult: float
        multiple to determine concavity. If concave up, conc_mult > 1. 
        If concave down, 0 < conc_mult < 1.
    
    Returns
    -------
    b2d: (N_draws,) numpy array
        distribution of second order coefficient in fitted polynomial
    
    b1d: (N_draws,) numpy array
        distribution of first order coefficient in fitted polynomial
    """
    
    T_1 = 3
    T_2 = 10

    N_draws = 10000
    D_1d = np.random.normal(loc=loc, scale=scale, size=N_draws)
    D_2d = D_1d * (T_2 * T_1**(-1)) * conc_mult

    b2d = (D_2d * T_1 - D_1d * T_2) * (T_1 * T_2 * (T_2 - T_1))**(-1)
    b1d = (D_1d * T_2**2 - D_2d * T_1**2) * (T_1 * T_2 * (T_2 - T_1))**(-1)
    
    print("First order mean: ", np.mean(b1d), 
      "\nSecond order mean: ", np.mean(b2d), 
      "\nFirst order std dev: ", np.std(b1d), 
      "\nSecond order std dev:", np.std(b2d))
    
    return b2d, b1d

def f(x, a, b):
    return a * x**2 + b * x

def g(x, a, b):
    return a * x + b

## Burke _et al._, 2018 damage functions

There is a damage function per SSP in this one, so it's a more involved process than the others.

In [ ]:
# mid century
burke_2049_ssp1 = np.genfromtxt(''.join([path, 'ssp1_2049_t_d.csv']), delimiter=',')
burke_2049_ssp1_t, burke_2049_ssp1_d = burke_2049_ssp1.T

burke_2049_ssp2 = np.genfromtxt(''.join([path, 'ssp2_2049_t_d.csv']), delimiter=',')
burke_2049_ssp2_t, burke_2049_ssp2_d = burke_2049_ssp2.T

burke_2049_ssp3 = np.genfromtxt(''.join([path, 'ssp3_2049_t_d.csv']), delimiter=',')
burke_2049_ssp3_t, burke_2049_ssp3_d = burke_2049_ssp3.T

burke_2049_ssp4 = np.genfromtxt(''.join([path, 'ssp4_2049_t_d.csv']), delimiter=',')
burke_2049_ssp4_t, burke_2049_ssp4_d = burke_2049_ssp4.T

burke_2049_ssp5 = np.genfromtxt(''.join([path, 'ssp5_2049_t_d.csv']), delimiter=',')
burke_2049_ssp5_t, burke_2049_ssp5_d = burke_2049_ssp5.T

# end of century
burke_2099_ssp1 = np.genfromtxt(''.join([path, 'ssp1_2099_t_d.csv']), delimiter=',')
burke_2099_ssp1_t, burke_2099_ssp1_d = burke_2099_ssp1.T

burke_2099_ssp2 = np.genfromtxt(''.join([path, 'ssp2_2099_t_d.csv']), delimiter=',')
burke_2099_ssp2_t, burke_2099_ssp2_d = burke_2099_ssp2.T

burke_2099_ssp3 = np.genfromtxt(''.join([path, 'ssp3_2099_t_d.csv']), delimiter=',')
burke_2099_ssp3_t, burke_2099_ssp3_d = burke_2099_ssp3.T

burke_2099_ssp4 = np.genfromtxt(''.join([path, 'ssp4_2099_t_d.csv']), delimiter=',')
burke_2099_ssp4_t, burke_2099_ssp4_d = burke_2099_ssp4.T

burke_2099_ssp5 = np.genfromtxt(''.join([path, 'ssp5_2099_t_d.csv']), delimiter=',')
burke_2099_ssp5_t, burke_2099_ssp5_d = burke_2099_ssp5.T

### SSP1

In [ ]:
burke_fit = curve_fit(g, burke_2049_ssp1_t, burke_2049_ssp1_d)
burke_means, burke_covars = burke_fit
burke_a_mean, burke_b_mean = burke_means

burke_fit_end = curve_fit(f, burke_2099_ssp1_t, burke_2099_ssp1_d)
burke_mean_end, burke_covars_end = burke_fit_end
burke_a_mean_end, burke_b_mean_end = burke_mean_end

print("Mid-century:")
burke_ssp1_mid_d2, burke_ssp1_mid_d1 = fitted_curve(0.075, 0.01, 2.5)

print("\nEnd-of-century:")
burke_ssp1_end_d2, burke_ssp1_end_d1 = fitted_curve(0.2, 0.04, .87)

temp = np.arange(0, 7, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp1_mid_d1[i] + burke_ssp1_mid_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp1_mid_d1) + np.mean(burke_ssp1_mid_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * burke_a_mean + burke_b_mean, linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2049_ssp1_t, burke_2049_ssp1_d)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp1_end_d1[i] + burke_ssp1_end_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp1_end_d1) + np.mean(burke_ssp1_end_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (burke_a_mean_end * temp + burke_b_mean_end), linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2099_ssp1_t, burke_2099_ssp1_d)

plt.ylim(0,0.7)

### SSP2

In [ ]:
burke_fit = curve_fit(g, burke_2049_ssp2_t, burke_2049_ssp2_d)
burke_means, burke_covars = burke_fit
burke_a_mean, burke_b_mean = burke_means

burke_fit_end = curve_fit(f, burke_2099_ssp2_t, burke_2099_ssp2_d)
burke_mean_end, burke_covars_end = burke_fit_end
burke_a_mean_end, burke_b_mean_end = burke_mean_end

print("Mid-century:")
burke_ssp2_mid_d2, burke_ssp2_mid_d1 = fitted_curve(0.065, 0.01, 2)

print("\nEnd-of-century:")
burke_ssp2_end_d2, burke_ssp2_end_d1 = fitted_curve(0.195, 0.04, .75)

temp = np.arange(0, 7, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp2_mid_d1[i] + burke_ssp2_mid_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp2_mid_d1) + np.mean(burke_ssp2_mid_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * burke_a_mean + burke_b_mean, linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2049_ssp2_t, burke_2049_ssp2_d)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp2_end_d1[i] + burke_ssp2_end_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp2_end_d1) + np.mean(burke_ssp2_end_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (burke_a_mean_end * temp + burke_b_mean_end), linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2099_ssp2_t, burke_2099_ssp2_d)

plt.ylim(0,0.7)

### SSP3

In [ ]:
burke_fit = curve_fit(g, burke_2049_ssp3_t, burke_2049_ssp3_d)
burke_means, burke_covars = burke_fit
burke_a_mean, burke_b_mean = burke_means

burke_fit_end = curve_fit(f, burke_2099_ssp3_t, burke_2099_ssp3_d)
burke_mean_end, burke_covars_end = burke_fit_end
burke_a_mean_end, burke_b_mean_end = burke_mean_end

print("Mid-century:")
burke_ssp3_mid_d2, burke_ssp3_mid_d1 = fitted_curve(0.062, 0.01, 2)

print("\nEnd-of-century:")
burke_ssp3_end_d2, burke_ssp3_end_d1 = fitted_curve(0.19, 0.04, .69)

temp = np.arange(0, 7, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp3_mid_d1[i] + burke_ssp3_mid_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp3_mid_d1) + np.mean(burke_ssp3_mid_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * burke_a_mean + burke_b_mean, linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2049_ssp3_t, burke_2049_ssp3_d)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp3_end_d1[i] + burke_ssp3_end_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp3_end_d1) + np.mean(burke_ssp3_end_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (burke_a_mean_end * temp + burke_b_mean_end), linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2099_ssp3_t, burke_2099_ssp3_d)

plt.ylim(0,0.7)

### SSP4

In [ ]:
burke_fit = curve_fit(g, burke_2049_ssp4_t, burke_2049_ssp4_d)
burke_means, burke_covars = burke_fit
burke_a_mean, burke_b_mean = burke_means

burke_fit_end = curve_fit(f, burke_2099_ssp4_t, burke_2099_ssp4_d)
burke_mean_end, burke_covars_end = burke_fit_end
burke_a_mean_end, burke_b_mean_end = burke_mean_end

print("Mid-century:")
burke_ssp4_mid_d2, burke_ssp4_mid_d1 = fitted_curve(0.049, 0.01, 2.5)

print("\nEnd-of-century:")
burke_ssp4_end_d2, burke_ssp4_end_d1 = fitted_curve(0.13, 0.04, .82)

temp = np.arange(0, 7, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp4_mid_d1[i] + burke_ssp4_mid_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp4_mid_d1) + np.mean(burke_ssp4_mid_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * burke_a_mean + burke_b_mean, linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2049_ssp4_t, burke_2049_ssp4_d)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp4_end_d1[i] + burke_ssp4_end_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp4_end_d1) + np.mean(burke_ssp4_end_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (burke_a_mean_end * temp + burke_b_mean_end), linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2099_ssp4_t, burke_2099_ssp4_d)

plt.ylim(0,0.7)

### SSP5

In [ ]:
burke_fit = curve_fit(g, burke_2049_ssp5_t, burke_2049_ssp5_d)
burke_means, burke_covars = burke_fit
burke_a_mean, burke_b_mean = burke_means

burke_fit_end = curve_fit(f, burke_2099_ssp5_t, burke_2099_ssp5_d)
burke_mean_end, burke_covars_end = burke_fit_end
burke_a_mean_end, burke_b_mean_end = burke_mean_end

print("Mid-century:")
burke_ssp5_mid_d2, burke_ssp5_mid_d1 = fitted_curve(0.065, 0.01, 2.1)

print("\nEnd-of-century:")
burke_ssp5_end_d2, burke_ssp5_end_d1 = fitted_curve(0.155, 0.04, .82)

temp = np.arange(0, 7, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp5_mid_d1[i] + burke_ssp5_mid_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp5_mid_d1) + np.mean(burke_ssp5_mid_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * burke_a_mean + burke_b_mean, linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2049_ssp5_t, burke_2049_ssp5_d)

for i in range(1000):
    plt.plot(temp, temp * (burke_ssp5_end_d1[i] + burke_ssp5_end_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(burke_ssp5_end_d1) + np.mean(burke_ssp5_end_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (burke_a_mean_end * temp + burke_b_mean_end), linestyle='dashed', color='r', linewidth=3)
plt.scatter(burke_2099_ssp5_t, burke_2099_ssp5_d)

plt.ylim(0,0.7)

## Structural damage function

In [ ]:
struct_data = np.genfromtxt("/data/keeling/a/adammb4/ClimateEcon/ez-climate/TCREZClimate/data/IPCC_WGII_RoseDamageEstimates.csv", delimiter=',', usecols=[0,1])

In [ ]:
struct_T, struct_dam = struct_data.T

In [ ]:
struct_fit = curve_fit(f, struct_T, struct_dam)
struct_means, struct_covars = struct_fit
struct_a_mean, struct_b_mean = struct_means

print("Structural df:")
struct_d2, struct_d1 = fitted_curve(0.027, 0.01, 2.8)

# double check fit (uncomment below)
dist_d2 = np.random.normal(loc=0.0023056898884004605, scale=0.0008591468977763477, size=1000)
dist_d1 = np.random.normal(loc=0.0020495021230226337, scale=0.0007636861313567542, size=1000)

temp = np.arange(0, 7, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (struct_d1[i] + struct_d2[i] * temp), alpha=0.01)
    #plt.plot(temp, temp * (dist_d1[i] + dist_d2[i] * temp), alpha=0.01, color='r')

plt.plot(temp, temp * (np.mean(struct_d1) + np.mean(struct_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (struct_a_mean* temp + struct_b_mean), linestyle='dashed', color='r', linewidth=3)
plt.scatter(struct_T, struct_dam)

## Meta-analytic damage function

In [ ]:
hs_data = np.genfromtxt("/data/keeling/a/adammb4/ClimateEcon/ez-climate/TCREZClimate/data/IPCC_WGII_HowardSternerDamageEstimates.csv", delimiter=',', usecols=[0,1])

In [ ]:
hs_T, hs_dam = hs_data.T

In [ ]:
hs_fit = curve_fit(f, hs_T, hs_dam)
hs_means, hs_covars = hs_fit
hs_a_mean, hs_b_mean = hs_means

print("Structural df:")
hs_d2, hs_d1 = fitted_curve(0.063, 0.022, 3.3)

temp = np.arange(0, 12, 0.01)

for i in range(1000):
    plt.plot(temp, temp * (hs_d1[i] + hs_d2[i] * temp), alpha=0.01)

plt.plot(temp, temp * (np.mean(hs_d1) + np.mean(hs_d2) * temp), color='k', linewidth=3)   
plt.plot(temp, temp * (hs_a_mean * temp + hs_b_mean), linestyle='dashed', color='r', linewidth=3)
plt.scatter(hs_T, hs_dam)